### Welche Wörter hatten wann Konjunktur?

Challenge: Die Wörter müssen nicht nur häufig vorkommen. Sie müssen auch über die Zeit variieren, sonst wirds langweilig.

Wie finde ich das heraus?? 

Ich brauche ein Dataframe mit folgenden Infos:

* Monat+Jahr (als Dateteime-Objekt)
* Wichtige Nomina-Counts ohne Füllwörter (die wurden vorher rausgefiltert)

Das ergibt in etwa folgendes Datenformat:
1. Spalte: Datetime.
2. Weitere Spalten: Nomina-Counts.

Dann aber: Wie finde ich heraus, welche variieren, welche nicht. Und vor allem, welche Interessant sind?

In [101]:
import spacy
from spacy.matcher import Matcher
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os

In [102]:
nlp = spacy.load("de_core_news_sm")
import de_core_news_sm
nlp = de_core_news_sm.load()

In [ ]:
import pandas as pd

### Fischen der Filterwörter pro Jahr

In der Liste keywords habe ich wichtige Wörter herausgeschrieben. Idee: Deren Auftreten relativ zur Gesamtzahl an Token je Textdokument zu einer SR-Sitzung angeben.

Das gibt dann eine Tabelle folgender Form:

|Monat Sitzung | Frequenz W1 | Frequenz W2 | ...

| Monat = ID | f(w1) | f(w2| ...

1. Datum herausfischen

In [4]:
folder_path = "/Users/j/Documents/GitHub/abschlussarbeit/pv_textfiles_de"
list_of_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

dates = []
for txt_file in list_of_files:
    file_path = os.path.join(folder_path, txt_file)
    file_name = os.path.basename(file_path)
    date_file = file_name[0:6]
    dates.append(date_file)

Wortfrequenzen herausfischen
Dafür brauchen wir:
* Anzahl herausgefischte token, len
* Anzahl herausgefischte Wörter aus keywords (manuell aus dem Explorations-Notebook herausgefischt)
* Um das einigermassen übersichtlich zu machen, brauchen wir eine Funktion, in der ein Text tokenisiert, lemmatisiert und die Substantive herausgefischt werden

In [217]:
%store keywords

Stored 'keywords' (list)


In [15]:
with open("/Users/j/Documents/GitHub/abschlussarbeit/pv_textfiles_de/2004-12SR-DE.txt", 'r') as file:
    # Read the contents of the file
    test_text = file.read()

In [78]:
folder_path = "/Users/j/Documents/GitHub/abschlussarbeit/pv_textfiles_de"
list_of_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

results=[]
lengths=[]
dates2=[]
for txt_file in list_of_files:
    file_path = os.path.join(folder_path, txt_file)
    file_name = os.path.basename(file_path)
    date_file = file_name[0:7]
    dates2.append(date_file)
    with open(file_path, 'r') as file:
        text = file.read()
        doc = nlp(text)
        lemmas_list = [str(token.lemma_) for token in doc]
        length = len(lemmas_list)
        lengths.append(length)
        
        element_counts = Counter()  # Initialize Counter outside the loop
        substantives = []
        
        for elem in lemmas_list:
            doc = nlp(elem)
            for token in doc:
                if "NOUN" in token.pos_:
                    substantives.append(str(token.lemma_))
        
        for elem in substantives:
            if elem in keywords:
                element_counts[elem] += 1  # Update the Counter
        
        result_dict = dict(element_counts)  # Convert Counter to a regular dictionary
        results.append(result_dict)

In [88]:
df = pd.DataFrame(results,index=dates2)

In [94]:
df.insert(0,"Anzahl Token",lengths)

In [96]:
df = df.fillna(0)

In [98]:
df.index = pd.to_datetime(df.index)
df=df.sort_index()
df

,Anzahl Token,Budget,Schachfestival,Eidgenos,Ökologie,Tradition,Autobahn,Pandemie,Kongresshaus,Individualverkehr,...,Wohnsitzzulag,Parkierungsanlag,Tierpark,Flüchtling,Agglolac,Klimaschutz,Eigentümerstrategie,Begleitgruppe,Filag,Baurecht
1999-01-01,18342,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999-02-01,20637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1999-03-01,37684,4.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999-04-01,30791,5.0,0.0,0.0,0.0,0.0,1.0,0.0,32.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1999-05-01,15557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01,8230,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-01,41746,13.0,0.0,7.0,0.0,1.0,0.0,16.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0
2023-08-01,14903,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2023-09-01,18832,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [209]:
df["Eidgenos"].sort_values(ascending=False).head(20)

2013-10-01    81.0
2020-06-01    60.0
2018-10-01    59.0
2019-06-01    55.0
2019-11-01    51.0
2013-03-01    50.0
2012-12-01    49.0
2019-03-01    46.0
2018-09-01    45.0
2014-10-01    43.0
2017-02-01    42.0
2021-01-01    39.0
2018-06-01    35.0
2013-06-01    33.0
2013-12-01    32.0
2015-08-01    32.0
2019-12-01    31.0
2012-06-01    30.0
2015-05-01    29.0
2016-02-01    29.0
Name: Eidgenos, dtype: float64

Jetzt exportieren. Einmal "sichern" dadurch, dass ich es als .csv abspeichere. Aber auch mit dem magic command.

In [99]:
df.to_csv("wordcount_dataframe_99-23.csv",index=True)

In [ ]:
%store df

### Aufgeteilt in drei Schritte:

* Tokenize
* Stemm
* get substantives

Das müssen wir jetzt für die special cases tun, und zwar für folgende Wortliste, die wir schon dem tokenizer übergeben müssen:

"Agglolac", "Westast", "Expo", "Expo 01", "Expo 02"



In [152]:
custom_words = ["Agglolac","agglolac", "Westast", "westast", "Expo", "expo", "expo 01", "expo 02", "Expo 01", "Expo 02"]

In [143]:
test_text="Jetzt schreibn wir \n mal n String mit Agglolac drn oder Expo 01 oder was mit dem Westast männlicher Art dann wurds doch Expo 02 aber auch nicht Expo 43 so geschehn würdn sollt,,achja und Westast Agglolac. hätt wärs nach denen da öben gegangen."

In [131]:
def tokenizer(text):
    words = word_tokenize(text)
    words_alpha = [word.lower() for word in words if word.isalpha()]  # Ignore case
    stop_words_german = stopwords.words('german')  # Use a different name for the variable
    result = [word for word in words_alpha if word not in stop_words_german]
    text_minus_stopwords = " ".join(result)
    return text_minus_stopwords

In [144]:
tokenizer(test_text)

'schreibn mal n string agglolac drn expo westast männlicher art wurds expo expo geschehn würdn sollt westast agglolac hätt wärs denen öben gegangen'

In [145]:
test_text_tokenized = tokenizer(test_text)
test_text_tokenized

'schreibn mal n string agglolac drn expo westast männlicher art wurds expo expo geschehn würdn sollt westast agglolac hätt wärs denen öben gegangen'

Fazit: Der Tokenizer erkennt die Dinger als token! Nun müssen wir sie spacy auch als lemmata und substantive verkaufen.

In [146]:
from spacy.matcher import Matcher
nlp = spacy.load("de_core_news_sm")

In [147]:
doc = nlp(test_text_tokenized)

In [148]:
# Initialize the Matcher with the shared vocabulary
matcher = Matcher(nlp.vocab)

In [151]:
# Create a pattern matching two tokens: "iPhone" and "X"
pattern = [[{"LEMMA": "expo"}],
           [{"LEMMA": "agglolac"}],
           [{"LEMMA":"westast"}]
             ]
# Add the custom lemma patterns to the matcher
for idx, pattern in enumerate(pattern, start=1):
    matcher.add("pattern_{}".format(idx), [pattern])

# Use the matcher on the doc
matches = matcher(doc)
print("Matches:", [doc[start:end].text for match_id, start, end in matches])

Matches: ['agglolac', 'expo', 'westast', 'expo', 'expo', 'westast', 'agglolac']


Funktioniert offensichtlich! Nun müssen wir die ganze Übung nochmals machen, allerdings, ohne sie nochmals als Substantive raufischen zu müssen, sondern wir tun das direkt mit matched = [doc[start:end].text for match_id, start, end in matches] und len(test_text_tokenized).

In [153]:
folder_path = "/Users/j/Documents/GitHub/abschlussarbeit/pv_textfiles_de"
list_of_files = [file for file in os.listdir(folder_path) if file.endswith(".txt")]

results=[]
lengths=[]
dates=[]
for txt_file in list_of_files:
    file_path = os.path.join(folder_path, txt_file)
    file_name = os.path.basename(file_path)
    date_file = file_name[0:7]
    dates2.append(date_file)
    with open(file_path, 'r') as file:
        text = file.read()
        doc = nlp(text)
        lemmas_list = [str(token.lemma_) for token in doc]
        length = len(lemmas_list)
        lengths.append(length)
        
        element_counts = Counter()  # Initialize Counter outside the loop       
        for elem in lemmas_list:
            if elem in custom_words:
                element_counts[elem] += 1  # Update the Counter
        
        result_dict = dict(element_counts)  # Convert Counter to a regular dictionary
        results.append(result_dict)

In [185]:
len(results)

255

In [186]:
dates = dates2[:255]

In [196]:
dates;

In [166]:
len(lengths)

255

In [192]:
df_spezial = pd.DataFrame(results,index=dates)

In [193]:
df_spezial

,Westast,Expo,Agglolac,westast,expo
2022-12,1.0,NaN,NaN,NaN,NaN
2011-10,NaN,NaN,NaN,NaN,NaN
2011-11,NaN,NaN,NaN,NaN,NaN
2000-04,NaN,NaN,NaN,NaN,NaN
2000-05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2023-02,NaN,NaN,NaN,NaN,NaN
2010-01,1.0,NaN,NaN,NaN,NaN
2004-08,NaN,3.0,NaN,NaN,NaN
2004-09,NaN,1.0,NaN,NaN,NaN


In [194]:
df_spezial = df_spezial.fillna(0)
df_spezial

,Westast,Expo,Agglolac,westast,expo
2022-12,1.0,0.0,0.0,0.0,0.0
2011-10,0.0,0.0,0.0,0.0,0.0
2011-11,0.0,0.0,0.0,0.0,0.0
2000-04,0.0,0.0,0.0,0.0,0.0
2000-05,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2023-02,0.0,0.0,0.0,0.0,0.0
2010-01,1.0,0.0,0.0,0.0,0.0
2004-08,0.0,3.0,0.0,0.0,0.0
2004-09,0.0,1.0,0.0,0.0,0.0


In [195]:
df_spezial["Anzahl Token2"] = df_spezial.insert(0,"Anzahl Token",lengths)
df_spezial

,Anzahl Token,Westast,Expo,Agglolac,westast,expo,Anzahl Token2
2022-12,28477,1.0,0.0,0.0,0.0,0.0,None
2011-10,32574,0.0,0.0,0.0,0.0,0.0,None
2011-11,8161,0.0,0.0,0.0,0.0,0.0,None
2000-04,19074,0.0,0.0,0.0,0.0,0.0,None
2000-05,19414,0.0,0.0,0.0,0.0,0.0,None
...,...,...,...,...,...,...,...
2023-02,13337,0.0,0.0,0.0,0.0,0.0,None
2010-01,18875,1.0,0.0,0.0,0.0,0.0,None
2004-08,20449,0.0,3.0,0.0,0.0,0.0,None
2004-09,17630,0.0,1.0,0.0,0.0,0.0,None


In [197]:
df_spezial.index = pd.to_datetime(df_spezial.index)
df_spezial=df_spezial.sort_index()
df_spezial

,Anzahl Token,Westast,Expo,Agglolac,westast,expo,Anzahl Token2
1999-01-01,18342,0.0,0.0,0.0,0.0,0.0,None
1999-02-01,20637,0.0,2.0,0.0,0.0,0.0,None
1999-03-01,37684,0.0,2.0,0.0,0.0,0.0,None
1999-04-01,30791,0.0,6.0,0.0,0.0,0.0,None
1999-05-01,15557,0.0,0.0,0.0,0.0,0.0,None
...,...,...,...,...,...,...,...
2023-05-01,8230,0.0,0.0,0.0,0.0,0.0,None
2023-06-01,41746,0.0,1.0,0.0,0.0,0.0,None
2023-08-01,14903,0.0,0.0,0.0,0.0,0.0,None
2023-09-01,18832,0.0,0.0,0.0,0.0,0.0,None


In [198]:
df_spezial = df_spezial.iloc[:, :-1]
df_spezial

,Anzahl Token,Westast,Expo,Agglolac,westast,expo
1999-01-01,18342,0.0,0.0,0.0,0.0,0.0
1999-02-01,20637,0.0,2.0,0.0,0.0,0.0
1999-03-01,37684,0.0,2.0,0.0,0.0,0.0
1999-04-01,30791,0.0,6.0,0.0,0.0,0.0
1999-05-01,15557,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
2023-05-01,8230,0.0,0.0,0.0,0.0,0.0
2023-06-01,41746,0.0,1.0,0.0,0.0,0.0
2023-08-01,14903,0.0,0.0,0.0,0.0,0.0
2023-09-01,18832,0.0,0.0,0.0,0.0,0.0


In [210]:
%store df_spezial

Stored 'df_spezial' (DataFrame)


In [200]:
df_spezial_relativ = df_spezial.copy()

In [204]:
df_spezial_relativ.iloc[:, 1:] = df_spezial_relativ.iloc[:, 1:].div(df_spezial["Anzahl Token"], axis=0)
df_spezial_relativ

,Anzahl Token,Westast,Expo,Agglolac,westast,expo
1999-01-01,18342,0.0,0.000000,0.0,0.0,0.0
1999-02-01,20637,0.0,0.000097,0.0,0.0,0.0
1999-03-01,37684,0.0,0.000053,0.0,0.0,0.0
1999-04-01,30791,0.0,0.000195,0.0,0.0,0.0
1999-05-01,15557,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...
2023-05-01,8230,0.0,0.000000,0.0,0.0,0.0
2023-06-01,41746,0.0,0.000024,0.0,0.0,0.0
2023-08-01,14903,0.0,0.000000,0.0,0.0,0.0
2023-09-01,18832,0.0,0.000000,0.0,0.0,0.0


In [205]:
df_spezial_relativ.to_csv('expo_westast_agglolac.csv', index=False)

In [206]:
%store df_spezial_relativ

Stored 'df_spezial_relativ' (DataFrame)


In [42]:
#stemmer(text_minus_stopwords)

In [31]:
lemmas_list = stemmer(text_minus_stopwords)

In [34]:
def get_substantives(list):
    substantives = []
    for elem in list:
        doc = nlp(elem)
        for token in doc:
            if "NOUN" in token.pos_:
                substantives.append(str(token.lemma_))
    return substantives

In [43]:
#get_substantives(lemmas_list)

Jetzt brauchen wir noch einen keyword-counter

In [36]:
from collections import Counter

In [37]:
def keyword_count(list,keywords):
    for elem in list:
        if elem in keywords:
            element_counts = Counter(list)
            result_dict = {element: element_counts[element] for element in keywords}
    return result_dict

In [38]:
keyword_count(lemmas_list,keywords)

{'Budget': 0,
 'Finanzkommission': 0,
 'Kosten': 6,
 'Arbeitszeit': 0,
 'Stadtordnung': 16,
 'Verkehr': 27,
 'Energie': 1,
 'Baum': 1,
 'Investition': 12,
 'Investitionsplanung': 12,
 'Eidgenosse': 0,
 'Eidgenos': 0,
 'Scherrer': 2,
 'Fehr': 10,
 'Steuer': 4,
 'Steuersatz': 0,
 'Umwelt': 1,
 'Auto': 8,
 'Integration': 0,
 'Wohnungsmarkt': 1,
 'Bahnhof': 4,
 'Bahnhofplatz': 1,
 'Kongresshaus': 0,
 'Polizei': 0,
 'Polizist': 0,
 'Neumarktplatz': 0,
 'Ordnung': 2,
 'Strom': 0,
 'Steuererhöhung': 0,
 'Steuersenkung': 0,
 'Finanzhaushalt': 0,
 'Bus': 4,
 'Wirtschaft': 2,
 'Geschlecht': 0,
 'Theater': 0,
 'Sozialhilfe': 0,
 'Sozialhilfequote': 0,
 'Finanzstrategie': 0,
 'Zentralplatz': 7,
 'Gas': 0,
 'Kreisel': 2,
 'Krieg': 0,
 'Budgetdebatte': 0,
 'Svp': 2,
 'Autobahn': 1,
 'Agglolac': 0,
 'Ehefrau': 6,
 'Zweisprachigkeit': 1,
 'Beleuchtung': 1,
 'Strandbad': 6,
 'Stellenschaffung': 0,
 'Stellenstreichung': 0,
 'Baurecht': 3,
 'Baurechtszins': 0,
 'Museum': 1,
 'Jugend': 0,
 'Freiheitsparte

Es könnte ja sein, dass in einer Sitzung ein Wort gar nicht vorkommt, dafür in anderen Sitzungen wichtig war. Wenn es wichtig war, dann fiel es pro Sitzung auch einige Male. Diese Wörter kommen dann pro Monat auf eine Longlist aller Wörter, die wir dann über alle Monate-Jahre betrachtet anschauen. 
Erste Aufgabe hier also:

* welche Wörter fallen gehäuft an?

Dann:

* Sammeln dieser Wörter aus allen Monaten-Jahren in einer grossen Liste

Dann: 

* Für alle Monate-Jahre: Inzidenz dieser Wörter, relativ zu len(substantive) oder len(text_de)?

In [72]:
from collections import Counter

In [73]:
word_counts = Counter(substantive)

# Print the most common words and their frequencies
print("Most Common Words:")
for word, count in word_counts.most_common(1000):  # Adjust the number as needed
    print(f"{word}: {count}")

Most Common Words:
Biel: 22
Jahr: 19
Kosten: 18
Forum: 11
Sap: 10
Abstimmung: 9
Betrag: 9
Gemeinderat: 8
Firma: 8
Kénn: 7
Strategie: 7
Moglichkeit: 7
Problem: 7
Erschliessungsfunktion: 7
Basiserschliessungsanlage: 7
Laubscherweg: 6
Stelle: 6
Projekt: 6
informatik: 6
Gemeinde: 6
Seite: 5
Initiative: 5
Franken: 5
Herr: 5
Stadtordnung: 5
Ehefrau: 5
Kind: 5
ei: 5
Konn: 5
Baudirektion: 5
Punkt: 5
IG: 5
Informatik: 5
Vernetzung: 5
Informatikstrategie: 5
Geld: 5
Einmundung: 5
Mehrbreite: 5
Wohneigentum: 4
Informatiklésung: 4
Software: 4
Basiserschliessung: 4
Uberbauungsordnung: 4
Ungefahr: 4
Bereich: 4
Prifung: 4
Aufgabe: 4
Geschafte: 4
Bericht: 4
Kredit: 4
Kontakt: 4
Zweisprachigkeit: 4
Sache: 4
Verwaltung: 4
Ordnung: 4
Konne: 4
Genossenschaft: 4
Scherrer: 3
Ausbildung: 3
Sicherheit: 3
Kreuzplatz: 3
Bedingung: 3
Verkehrsmittel: 3
Investitionsbeitrag: 3
Umbau: 3
Einkommen: 3
Leistung: 3
Stadtratsprasident: 3
Schulerin: 3
Stimmzettel: 3
--: 3
Lehrlinge: 3
Mitarbeiter: 3
Polizei: 3
re: 3
Kénnen